In [1]:
# This version uses a model architecture with two LSTM layers and a dropout layer in between
# again, just using part1 data for training and validation

# Here I will be building out the architecture of the first classification LSTM
# At each time step, this LSTM will take in a vector representing the extracted audio and visual features from Liris-ACCEDE
# Its goal is to output whether or not the movie induces fear at each time step


# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [3]:
# uploading the X values
X_input = load_Xinput(get_fc6_directory(7))
print(X_input.shape)

(212, 4096)


In [4]:
# uploading the Y values
y_data_input = fear_oneHot(212, 'fear_annotations_part01/MEDIAEVAL18_7_Fear.txt')
print(y_data_input.shape)

(212,)


In [5]:
# Uploading part01 of the local fc6 data

timesteps = 212   # the number of seconds in movie 07 --> i will figure out how to programmatically get this value
data_dim = 4096    # the number of output values from VGG16 layer fc6 --> switch to programmatic later
num_movies = 4
batch_size = 7
num_epochs = 5
validation_num = 3

# set up the X_train_data master array
X_train_data = np.zeros([num_movies, timesteps, data_dim]) #oooooh this array will have to be as long as the longest
                                                            # movie and padded with zeros --> this won't cause problems
                                                            # right?
X_valid_data = np.zeros([validation_num, timesteps, data_dim])
        
# for each movie number between and including 7 and 13
for num in range(num_movies):
    # load the X_input data
    X_input = load_Xinput(get_fc6_directory(7 + num))
    # put this X_input data into the X_train_data array
    X_train_data[num, :, :] = X_input
print(X_train_data.shape)



(4, 212, 4096)


In [6]:
# loading X validation set
X_valid_data = np.zeros([validation_num, timesteps, data_dim])

for num in range(validation_num):
    # load the X_input data
    X_valid = load_Xinput(get_fc6_directory(7 + num_movies + num))
    # put this X_input data into the X_train_data array
    X_valid_data[num, :, :] = X_valid
print(X_valid_data.shape)


(3, 212, 4096)


In [7]:
# uploading the y data

# set up y_train_data master array
Y_train_data = np.zeros([num_movies, timesteps])

# for each movie number between and including 7 and 13
for num in range(num_movies):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_train_data[num, :] = y_data
print(Y_train_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
# upload the Y validation set
Y_valid_data = np.zeros([validation_num, timesteps])

# for each movie number in validation set
for num in range(validation_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+ num_movies + num))
    # create a one-hot vector
    y_valid = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_valid_data[num, :] = y_valid
print(Y_valid_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
# model architecture
model = Sequential()
model.add(LSTM(212, return_sequences=True,
               input_shape=(timesteps, data_dim)))
# dropout layer
model.add(Dropout(0.5))

# another LSTM layer
model.add(LSTM(212, return_sequences=True))

# necessary flatten layer
model.add(Flatten()) 

# add the final dense layer and then softmax
model.add(Dense(212, activation='softmax'))

In [10]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [15]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [20]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 100, validation_data=(X_valid_data, Y_valid_data))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/100
4/4 [==============================] - 4s 894ms/step - loss: 0.8194 - binary_accuracy: 0.8066 - FScore2: 0.1389 - val_loss: 1.6816 - val_binary_accuracy: 0.7689 - val_FScore2: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 3s 773ms/step - loss: 0.8150 - binary_accuracy: 0.8054 - FScore2: 0.0000e+00 - val_loss: 1.7320 - val_binary_accuracy: 0.7689 - val_FScore2: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 4s 883ms/step - loss: 0.8055 - binary_accuracy: 0.8054 - FScore2: 0.0000e+00 - val_loss: 1.7915 - val_binary_accuracy: 0.7689 - val_FScore2: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 3s 769ms/step - loss: 0.8017 - binary_accuracy: 0.8066 - FScore2: 0.1389 - val_loss: 1.7218 - val_binary_accuracy: 0.7689 - val_FScore2: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 3s 794ms/step - loss: 0.8024 - binary_accuracy: 0.8054 - FScore2: 0.0000e+00 - val_loss: 1.8761 - val_b

In [19]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

model prediction:
[5.49479864e-07 5.93944719e-07 5.78420270e-07 5.83663393e-07
 5.65004484e-07 5.71319617e-07 5.68758537e-07 5.76131754e-07
 5.76161938e-07 5.51805670e-07 5.70887153e-07 5.70346799e-07
 5.73387240e-07 5.64274842e-07 5.55580129e-07 5.73830334e-07
 5.76743560e-07 5.86028818e-07 5.42460668e-07 5.77197000e-07
 5.66496851e-07 5.77163974e-07 5.84629390e-07 5.46381159e-07
 5.39094572e-07 5.61853597e-07 1.23930571e-04 1.49139509e-04
 1.52143548e-04 1.13163369e-04 1.36380404e-04 1.29767621e-04
 1.20852841e-04 1.01186968e-04 1.58411698e-04 1.22663376e-04
 1.17308482e-04 3.56533419e-04 3.32202122e-04 1.16111369e-04
 1.06199841e-04 1.19876968e-04 1.19230841e-04 1.61146963e-04
 1.27709907e-04 8.63752794e-05 1.47213388e-04 1.22408324e-04
 1.53755987e-04 1.18274947e-04 1.46849794e-04 1.04359366e-04
 4.71320556e-04 3.61709448e-04 4.56058217e-04 4.02220903e-04
 3.59833997e-04 4.11138346e-04 3.47381196e-04 3.03643988e-04
 1.20189186e-04 1.42655670e-04 1.19914235e-04 1.07191670e-04
 2.553

In [14]:
#try visualizing this model at some point?